In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R


def cam_calib(w, h, square_size, criteria, path,outpath):
    """
    说明：用来获得相机内参
    参数：
    w,h 棋盘格内角点个数
    square_size:棋盘格尺寸
    criteria：检测角点准则
    path：检测图像路径

    """
    list_path = os.listdir(path)

    # 准备棋盘格的世界坐标点
    objp = np.zeros((w * h, 3), np.float32)
    objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
    objp *= square_size

    grays = []
    objpoints = []  # 世界坐标点
    imgpoints = []  # 图像像素点

    # 遍历图像并检测棋盘角点
    for file in list_path:
        file_name = os.path.join(path, file)
        img = cv2.imread(file_name)
        scale = 0.5  # 50%缩小
        img_resize = cv2.resize(
            img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))

        gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
        # gray_resize = cv2.resize(gray, (int(gray.shape[1] * scale), int(gray.shape[0] * scale)))
        print('image pixel', gray.shape[::-1])
        grays.append(gray)

        ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
        print(f"Chessboard found in {file}: {ret}")

        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                        criteria)
            imgpoints.append(corners2)
            
            
            # 可视化检测到的角点（用 matplotlib 替代 OpenCV）
            img_with_corners = img_resize.copy()
            cv2.drawChessboardCorners(img_with_corners, (w, h), corners2, ret)
            cv2.imwrite(f'{outpath}\\{file}', img_with_corners)
            #使用 matplotlib 显示
            # plt.figure(figsize=(8, 6))
            # plt.imshow(cv2.cvtColor(img_with_corners, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
            # plt.scatter(corners2[:, 0, 0], corners2[:, 0, 1], marker='o', c='red', s=10, label="Detected Corners")  # `s`控制点大小
            # plt.title(f"Detected Corners: {file}")
            # plt.rcParams['lines.linewidth'] = 2  # 设置线条宽度
            # plt.axis("off")  # 隐藏坐标轴
            # plt.show()


    # 相机标定
    print("\nStarting camera calibration...")
    # ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    #     objpoints, imgpoints, grays[0].shape[::-1], None, None
    # )
    ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        grays[0].shape[::-1],
        None,
        None,
        flags=cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4
        | cv2.CALIB_FIX_K5)

    # 计算重投影误差
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i],
                                          camera_matrix, distortion_coeffs)
        error = cv2.norm(imgpoints[i].squeeze(), imgpoints2.squeeze(),
                         cv2.NORM_L2) / len(imgpoints2)
        mean_error += error

    print("\n总平均误差 (Mean Reprojection Error):", mean_error / len(objpoints))

    new_camera_matrix = camera_matrix.copy()
    new_camera_matrix[0, 0] /= scale  # fx
    new_camera_matrix[1, 1] /= scale  # fy
    new_camera_matrix[0, 2] /= scale  # cx
    new_camera_matrix[1, 2] /= scale  # cy
    # 打印结果
    print("相机矩阵 (Camera Matrix):\n", new_camera_matrix)
    print("\n畸变系数 (Distortion Coefficients):\n", distortion_coeffs)
    # 保存标定结果
    np.savez("calibration_result.npz",
             camera_matrix=new_camera_matrix,
             distortion_coeffs=distortion_coeffs,
             rvecs=rvecs,
             tvecs=tvecs)
    for file in list_path:
        file_name = os.path.join(path, file)
        img = cv2.imread(file_name)
        
    return new_camera_matrix, distortion_coeffs

In [3]:
def image_points_extract(image_path, w, h, criteria):
    """
    说明：从图片提取棋盘格点的像素坐标
    输入参数：
    image_path：图片路径
    w,h 棋盘格内角点个数
    criteria：检测角点准则
    outpath: 可视化角点图片输出路径
    """
    list_image_path = os.listdir(image_path)
    
    board_corners = []
    for file in list_image_path:
        file_name = os.path.join(image_path, file)
        img = cv2.imread(file_name)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
        if ret:
            # 通过角点亚像素精确化，提高角点位置精度
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                        criteria)
            board_corners.append(corners2)
            # 可视化角点
            # img_with_corners = cv2.drawChessboardCorners(
            #     img, (w, h), corners2, ret)
            # cv2.imwrite(f'{outpath}\\{file}', img_with_corners)
            print('finish corner extract')
        else:
            print("Chessboard corners not found.")
    return board_corners
                           

In [13]:
def object_points_calculate(T_ref, T_pose, point_board):
    """
    说明：计算某一位姿下棋盘格点在参考坐标系下的3d坐标
    T_ref:参考坐标系相对于base的变换矩阵
    T_pose该位姿下tcp相对于base的变换矩阵
    point_board：棋盘格点在tcp坐标系里的坐标
    """
    # 将3D点转换为齐次坐标
    origin_points_h = np.hstack(
        [point_board, np.ones((point_board.shape[0], 1))])
    # 使用矩阵乘法进行坐标变换
    #去掉齐次坐标中的最后一列，转换回普通的3D坐标
    object_points_base_inv = np.dot(T_pose, origin_points_h.T)
    object_points_o1 = np.dot(np.linalg.inv(T_ref), object_points_base_inv).T
    return object_points_o1[:, :3]


def kuka_to_transformation(pose):
    """将 KUKA 的点位信息转换为 4x4 变换矩阵"""
    # 平移向量
    t = np.array([pose[0], pose[1], pose[2]])

    # 计算旋转矩阵（ZYX顺序）
    rotation = R.from_euler('ZYX', [pose[3], pose[4], pose[5]],
                            degrees=True)  # KUKA的ABC通常以度为单位
    R_matrix = rotation.as_matrix()
    # print(R_matrix)

    # 构造4x4变换矩阵
    T = np.eye(4)
    T[:3, :3] = R_matrix
    T[:3, 3] = t
    # print('T', T)
    return T


def board_points_calculate(grid_size, grid_extend, grid_extend_j):
    """输出棋盘格点，由于4*4的图片检测出的格点顺序不同，所以横竖两种情况"""
    #格点竖着排
    points_col = np.array([(i * grid_size, j * grid_size, 0)
                           for i in grid_extend_j for j in grid_extend])
    #格点横着排
    points_row = np.array([(i * grid_size, j * grid_size, 0)
                           for j in grid_extend for i in grid_extend_j])
    print(points_row)
    return points_row


def cam_solve( object_points, image_points, camera_matrix,
              distortion_coeffs):
    """调用pnp函数，返回rvec,tvec"""
    _, rvec, tvec, inliners = cv2.solvePnPRansac(object_points,
                                                 image_points.squeeze(),
                                                 camera_matrix,
                                                 distortion_coeffs,
                                                 reprojectionError=8.0,
                                                 confidence=0.99,
                                                 iterationsCount=150)
    if not _:
        print("PnP 求解失败")
    else:
        return rvec, tvec


def process_cam_solve(ref, pose_list, grid_size, grid_extend, grid_extend_j,
                      translation_vec, image_path, w2, h2, criteria,camera_matrix, distortion_coeffs):
    """遍历四个不同的姿态，算出四组外参"""
    rvec_list = []
    tvec_list = []
    
    
    
    # 提取所有图像的角点
    image_points = image_points_extract(image_path, w2, h2, criteria)
    
    # 计算参考坐标系变换矩阵
    T_ref = kuka_to_transformation(ref)
    
    for i in range(len(pose_list)):
        # 计算棋盘格 3D 点（世界坐标系下）
        board_points = board_points_calculate(grid_size, grid_extend, grid_extend_j) + translation_vec
        
        # KUKA 机器人位姿变换矩阵
        T_pose = kuka_to_transformation(pose_list[i])
        
        # 计算棋盘格点在参考坐标系下的 3D 位置
        object_points = np.array(object_points_calculate(T_ref, T_pose, board_points), dtype=np.float64).reshape(-1, 1, 3)
        
        # 获取当前帧的角点
        image_points_curr = np.array(image_points[i], dtype=np.float64).reshape(-1, 2)
        
        # 计算相机位姿
        rvec, tvec = cam_solve(object_points, image_points_curr, camera_matrix, distortion_coeffs)
        rvec_list.append(rvec)
        tvec_list.append(tvec)

        # 计算旋转矩阵
        rotation_matrix, _ = cv2.Rodrigues(rvec)
        print("Rotation Matrix:\n", rotation_matrix)
        print("Translation Vector:\n", tvec)

        # 进行重投影计算
        img_points2, _ = cv2.projectPoints(object_points, rvec, tvec, camera_matrix, distortion_coeffs)
        img_points2 = np.array(img_points2, dtype=np.float64).reshape(-1, 2)

        # 计算重投影误差
        error = cv2.norm(image_points_curr, img_points2, cv2.NORM_L2) / len(img_points2)
        print("重投影误差 (Mean Reprojection Error):", error)

    return rvec_list, tvec_list


In [5]:
def re_project(corner, translation_vec,image,ref,pose,rvec,tvec,camera_matrix, distortion_coeffs,output_path):
    T_ref = kuka_to_transformation(ref)
    T_pose = kuka_to_transformation(pose)
    point_board = corner+ translation_vec
    object_points_re = object_points_calculate(T_ref,T_pose,point_board)
    project_corner_points2, _ = cv2.projectPoints(object_points_re, rvec, tvec, camera_matrix, distortion_coeffs)
    pts = np.array(project_corner_points2, dtype=np.int32)
    # 在图像上绘制四边形
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=5)

    # 保存结果
    cv2.imwrite(output_path, image)

In [8]:

# 计算相机内参
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100,
            0.0001)



camera_matrix, distortion_coeffs = cam_calib(
    w=9,
    h=11,
    square_size=45,
    criteria=criteria,
    path='D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\chessboard4calib',
    outpath='D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\chessboard_corner')




image pixel (2736, 1824)
Chessboard found in IMG_6575.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6576.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6577.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6578.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6579.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6580.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6581.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6582.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6583.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6584.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6585.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6586.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6587.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6588.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6589.JPG: True

Starting camera calibration...

总平均误差 (

In [26]:
ref=[-170, -1800, 1500, 58, 0, 90]#tcp在world里的位置
translation_vec = np.array([22.054, -4.485, -63.792])  #棋盘格中心在tcp坐标系里的位置
#
# translation_vec = np.array([50,-20,-20])
rvec_list, tvec_list = process_cam_solve(
    ref=ref,
    pose_list=[
               [1537.49, -2774.10, 1491.28, 65.12, -1.99, 100.67],
               [1537.49, -2813.63, 1483.98, 61.68, -5.31, 94.55],
               [1499.69, -2813.63, 1465.46, 61.39, -0.8, 97.98],
               [1499.69, -2813.63, 1465.46, 76.1, -2.81, 97.51],
               [1499.69,-2902.67,1465.46,76.1,-2.81,97.51],
[1499.69,-2902.67,1465.46,78.55,-3.13,101.83]],

    grid_size=90,
    grid_extend=[i for i in range(5, -6, -1)],
    grid_extend_j=[i for i in range(4, -5, -1)],
    translation_vec=translation_vec,
    image_path=
    'D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\image4camsolve',
    w2=9,
    h2=11,
    criteria=criteria,
    camera_matrix=camera_matrix,
    distortion_coeffs=distortion_coeffs)

for i in range(len(rvec_list)):
    re_project(
        corner=np.array(
            [[90, 90, 0], [90, -90, 0], [-90, -90, 0], [-90, 90, 0]],
            dtype=np.float64),
        translation_vec=translation_vec,
        image=cv2.imread(
            'D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\testimage\\IMG_6595.JPG'
        ),
        ref=ref,
        # pose=[-222.92, -1884.805, 1400, 32.616, 12.209, 70.802],
        pose=[-222.92, -1884.805, 1450, 32.616, 12.209, 70.802],rvec=rvec_list[i],
        tvec=tvec_list[i],
        camera_matrix=camera_matrix,
        distortion_coeffs=distortion_coeffs,
        output_path=
        f'D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\testimage\\draw_IMG_6595_{i}.JPG'
    )

finish corner extract
finish corner extract
finish corner extract
finish corner extract
finish corner extract
finish corner extract
[[ 360  450    0]
 [ 270  450    0]
 [ 180  450    0]
 [  90  450    0]
 [   0  450    0]
 [ -90  450    0]
 [-180  450    0]
 [-270  450    0]
 [-360  450    0]
 [ 360  360    0]
 [ 270  360    0]
 [ 180  360    0]
 [  90  360    0]
 [   0  360    0]
 [ -90  360    0]
 [-180  360    0]
 [-270  360    0]
 [-360  360    0]
 [ 360  270    0]
 [ 270  270    0]
 [ 180  270    0]
 [  90  270    0]
 [   0  270    0]
 [ -90  270    0]
 [-180  270    0]
 [-270  270    0]
 [-360  270    0]
 [ 360  180    0]
 [ 270  180    0]
 [ 180  180    0]
 [  90  180    0]
 [   0  180    0]
 [ -90  180    0]
 [-180  180    0]
 [-270  180    0]
 [-360  180    0]
 [ 360   90    0]
 [ 270   90    0]
 [ 180   90    0]
 [  90   90    0]
 [   0   90    0]
 [ -90   90    0]
 [-180   90    0]
 [-270   90    0]
 [-360   90    0]
 [ 360    0    0]
 [ 270    0    0]
 [ 180    0    0]
 [  

In [23]:
grid_size=90
grid_extend_j=[i for i in range(4,-5,-1)]

grid_extend=[i for i in range(5,-6,-1)]
print(grid_extend)
print(grid_extend_j)
points_row = np.array([(i * grid_size, j * grid_size, 0)
                           for j in grid_extend for i in grid_extend_j])
print(points_row)

[5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5]
[4, 3, 2, 1, 0, -1, -2, -3, -4]
[[ 360  450    0]
 [ 270  450    0]
 [ 180  450    0]
 [  90  450    0]
 [   0  450    0]
 [ -90  450    0]
 [-180  450    0]
 [-270  450    0]
 [-360  450    0]
 [ 360  360    0]
 [ 270  360    0]
 [ 180  360    0]
 [  90  360    0]
 [   0  360    0]
 [ -90  360    0]
 [-180  360    0]
 [-270  360    0]
 [-360  360    0]
 [ 360  270    0]
 [ 270  270    0]
 [ 180  270    0]
 [  90  270    0]
 [   0  270    0]
 [ -90  270    0]
 [-180  270    0]
 [-270  270    0]
 [-360  270    0]
 [ 360  180    0]
 [ 270  180    0]
 [ 180  180    0]
 [  90  180    0]
 [   0  180    0]
 [ -90  180    0]
 [-180  180    0]
 [-270  180    0]
 [-360  180    0]
 [ 360   90    0]
 [ 270   90    0]
 [ 180   90    0]
 [  90   90    0]
 [   0   90    0]
 [ -90   90    0]
 [-180   90    0]
 [-270   90    0]
 [-360   90    0]
 [ 360    0    0]
 [ 270    0    0]
 [ 180    0    0]
 [  90    0    0]
 [   0    0    0]
 [ -90    0    0]
 [-180    0

In [17]:
T_tcp = kuka_to_transformation([-22.45, 6.745, 641.177, 0, 90, 0])
T_center = kuka_to_transformation([-86.242, 2.26, 619.123, 0, 90, 0])
T_c2t = np.dot(np.linalg.inv(T_tcp), T_center)
print(T_c2t)

[[  1.      0.      0.     22.054]
 [  0.      1.      0.     -4.485]
 [  0.      0.      1.    -63.792]
 [  0.      0.      0.      1.   ]]


3